<a href="https://colab.research.google.com/github/fecsaba/automatic_photo/blob/main/auto_get_meta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Drive**

In [ ]:
!pip install transformers keybert
import os

# Csatoljuk a Google Drive-ot
from google.colab import drive
drive.mount('/content/drive')
!pip install pypng
!pip install Pillow

# ***Action***

```
'/content/drive/MyDrive/Images'
```



## Meta kiszedése

In [ ]:
from PIL import PngImagePlugin, Image
import os
import json

class PngValidator:
    """PNG fájlok érvényességének ellenőrzésére szolgáló osztály.

    Ez az osztály felelős a PNG fájlok formátumának validálásáért. Ellenőrzi, hogy egy fájl
    valóban érvényes PNG formátumú-e, függetlenül a fájl kiterjesztésétől.

    Methods:
        is_valid_png(file_path): Ellenőrzi egy fájl PNG formátumát.
    """

    @staticmethod
    def is_valid_png(file_path):
        """Ellenőrzi, hogy a fájl valóban PNG formátumú-e.

        A metódus megnyitja a fájlt és ellenőrzi annak belső formátumát,
        nem csak a kiterjesztést vizsgálja.

        Args:
            file_path (str): A vizsgálandó fájl teljes elérési útja.

        Returns:
            bool: True ha a fájl valódi PNG formátumú, False egyébként.

        Example:
            >>> validator = PngValidator()
            >>> is_valid = validator.is_valid_png("kep.png")
            >>> print(is_valid)
            True
        """
        try:
            with Image.open(file_path) as img:
                return isinstance(img, PngImagePlugin.PngImageFile) and img.format == 'PNG'
        except Exception:
            return False

class PngFileHandler:
    """PNG fájlok és mappák kezelésére szolgáló osztály.

    Ez az osztály felelős a fájlrendszer műveletek végrehajtásáért,
    beleértve a mappák beolvasását és a PNG fájlok azonosítását.

    Attributes:
        directory_path (str): A feldolgozandó mappa elérési útja.

    Methods:
        set_directory_path(directory_path): Beállítja a feldolgozandó mappa útvonalát.
        get_png_files(): Összegyűjti az érvényes PNG fájlokat a megadott mappából.
    """

    def __init__(self, directory_path=None):
        """Inicializálja a PngFileHandler osztályt.

        Args:
            directory_path (str, optional): A feldolgozandó mappa elérési útja.
                                          Alapértelmezett érték: None.
        """
        self.directory_path = directory_path

    def set_directory_path(self, directory_path):
        """Beállítja a feldolgozandó mappa útvonalát.

        Args:
            directory_path (str): A feldolgozandó mappa elérési útja.
        """
        self.directory_path = directory_path

    def get_png_files(self):
        """Összegyűjti az érvényes PNG fájlokat a megadott mappából.

        A metódus ellenőrzi a megadott mappában található összes fájlt,
        és csak azokat a fájlokat adja vissza, amelyek valóban PNG formátumúak.

        Returns:
            list: Érvényes PNG fájlok teljes elérési útjainak listája.

        Note:
            - Ha nincs megadva mappa útvonal, üres listával tér vissza
            - Ha a mappa nem létezik, üres listával tér vissza
            - Figyelmeztetést ad ki, ha talál olyan fájlt, ami .png kiterjesztésű,
              de nem valódi PNG formátumú
        """
        if not self.directory_path:
            print("Nincs megadva mappa útvonal!")
            return []

        if not os.path.exists(self.directory_path):
            print(f"A megadott mappa nem létezik: {self.directory_path}")
            return []

        png_files = []
        for filename in os.listdir(self.directory_path):
            file_path = os.path.join(self.directory_path, filename)
            if PngValidator.is_valid_png(file_path):
                png_files.append(file_path)
            elif filename.lower().endswith('.png'):
                print(f"\nFigyelmeztetés: A fájl '.png' kiterjesztésű, de nem valódi PNG: {filename}")

        return png_files

class MetadataReader:
    """PNG fájlok metaadatainak olvasására szolgáló osztály.

    Ez az osztály felelős a PNG fájlok metaadatainak kinyeréséért és
    megjelenítéséért. A metaadatok között szerepelhetnek például a készítő,
    a létrehozás dátuma, és egyéb egyedi információk.

    Methods:
        read_metadata(file_path): Beolvassa és megjeleníti egy PNG fájl metaadatait.
    """

    @staticmethod
    def read_metadata(file_path):
        """Beolvassa egy PNG fájl metaadatait.

        Args:
            file_path (str): A PNG fájl teljes elérési útja.

        Returns:
            dict: A metaadatok szótár formátumban, ahol a kulcsok a metaadat mezők nevei,
                 az értékek pedig a hozzájuk tartozó értékek.
                 Hiba esetén None-nal tér vissza.
                 Ha nincsenek metaadatok, üres szótárral tér vissza.

        Note:
            A metódus a következő eseteket kezeli:
            - Sikeres beolvasás esetén kiírja a metaadatokat
            - Ha nincsenek metaadatok, ezt jelzi
            - Ha a fájl nem található, hibaüzenetet ad
            - Egyéb hibák esetén hibaüzenetet ad
        """
        try:
            with Image.open(file_path) as img:
                metadata = img.text
                if metadata:
                    print(f"\nMetaadatok a következő fájlhoz: {os.path.basename(file_path)}")
                    print("-" * 50)
                    for k, v in metadata.items():
                        print(f"**{k}**:\n{v}\n")
                    return metadata
                else:
                    print(f"\nNem találtunk metaadatokat a következő fájlban: {os.path.basename(file_path)}")
                    return {}

        except FileNotFoundError:
            print(f"A fájl nem található: {file_path}")
            return None
        except Exception as e:
            print(f"Hiba történt a következő fájlnál {os.path.basename(file_path)}: {str(e)}")
            return None

class PngMetadataProcessor:
    """PNG fájlok metaadatainak feldolgozására szolgáló fő osztály.

    Ez az osztály koordinálja a teljes feldolgozási folyamatot, összekapcsolva
    a fájlkezelést és a metaadat-olvasást. Egy megadott mappában található összes
    PNG fájlt feldolgozza és összegyűjti azok metaadatait.

    Attributes:
        file_handler (PngFileHandler): A fájlkezelésért felelős objektum.

    Methods:
        process_directory(): Feldolgozza a megadott mappában található összes PNG fájlt.
        save_results_to_json(results): Elmenti az eredményeket JSON fájlba.
    """

    def __init__(self, directory_path=None):
        """Inicializálja a PngMetadataProcessor osztályt.

        Args:
            directory_path (str, optional): A feldolgozandó mappa elérési útja.
                                          Alapértelmezett érték: None.
        """
        self.file_handler = PngFileHandler(directory_path)

    def process_directory(self):
        """Feldolgozza a megadott mappában található összes PNG fájlt.

        A metódus végigmegy a megadott mappában található összes PNG fájlon,
        és összegyűjti azok metaadatait. A folyamat során:
        1. Azonosítja az érvényes PNG fájlokat
        2. Beolvassa minden fájl metaadatait
        3. Összesíti az eredményeket

        Returns:
            dict: Egy szótár, ahol a kulcsok a fájlnevek, az értékek pedig
                 a hozzájuk tartozó metaadatok szótár formátumban.
        """
        print(f"\nPNG fájlok keresése a következő mappában: {self.file_handler.directory_path}")
        print("=" * 70)

        png_files = self.file_handler.get_png_files()
        results = {}

        for file_path in png_files:
            metadata = MetadataReader.read_metadata(file_path)
            if metadata:
                results[os.path.basename(file_path)] = metadata

        if not results:
            print("\nNem találtunk érvényes PNG fájlokat a megadott mappában.")

        return results

    def save_results_to_json(self, results):
        """Elmenti az eredményeket JSON fájlba.

        Args:
            results (dict): A mentendő metaadatok szótár formátumban.

        Note:
            A fájl neve 'png_metadata_results.json' lesz, és az Image mappába kerül mentésre.
            Ha a fájl már létezik, felülírja azt.
        """
        if not results:
            print("\nNincs menthető adat.")
            return

        output_path = os.path.join(self.file_handler.directory_path, "png_metadata_results.json")
        try:
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(results, f, indent=2, ensure_ascii=False)
            print(f"\nAz eredmények sikeresen mentve: {output_path}")
        except Exception as e:
            print(f"\nHiba történt a fájl mentése közben: {str(e)}")

# Példa használat és tesztelés
if __name__ == "__main__":
    # Példa a program használatára
    processor = PngMetadataProcessor("/content/drive/MyDrive/Image/")
    all_metadata = processor.process_directory()

    # Eredmények mentése JSON fájlba
    processor.save_results_to_json(all_metadata)

    # Összesítés
    if all_metadata:
        print("\nÖsszesítés:")
        print("=" * 70)
        print(f"Összesen {len(all_metadata)} érvényes PNG fájl metaadatait olvastuk be.")

## Pozitív prompt kiszedése

### Leírás kiválasztása

In [ ]:
import json
import re
import sys

def extract_pos_from_parameters(input_json_path, output_json_path):
    """
    Extracts the 'Pos' value from the 'parameters' string in the input JSON and saves it to a new JSON file.
    """
    try:
        with open(input_json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"Error: Input file not found at {input_json_path}")
        return False
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON in input file {input_json_path}")
        return False
    except Exception as e:
        print(f"Error: An unexpected error occurred: {type(e).__name__} - {e}")
        return False

    pos_data = {}
    pos_pattern = re.compile(r'Pos:\s*([\s\S]*?)(?:,\n|$)')  # Regex pattern to extract the Pos value

    for image_filename, image_data in data.items():
        if 'parameters' in image_data and isinstance(image_data['parameters'], str):
            match = pos_pattern.search(image_data['parameters'])
            if match:
                pos_value = match.group(1).strip()  # Get the matched Pos value and strip whitespace
                pos_data[image_filename] = pos_value
            else:
                print(f"Warning: No 'Pos' key found in parameters for {image_filename}")
                pos_data[image_filename] = ""  # Üres string, ha nincs 'Pos'
        else:
            print(f"Warning: No 'parameters' string found for {image_filename}")
            pos_data[image_filename] = ""  # Üres string, ha nincs 'parameters'

    try:
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump(pos_data, f, indent=2)  # Szebb JSON formázás
    except Exception as e:
        print(f"Error: Could not write to output file {output_json_path}: {e}")
        return False

    return True

if __name__ == '__main__':
    if len(sys.argv) != 3:
        print("Usage: python3 extract_pos.py <input_json_file> <output_json_file>")
        sys.exit(1)

    input_file = "/content/drive/MyDrive/Image/png_metadata_results.json"
    output_file = "/content/drive/MyDrive/Image/desc.json"

    if extract_pos_from_parameters(input_file, output_file):
        print(f"Successfully extracted 'Pos' values to {output_file}")
    else:
        print("Extraction failed.")

### Szöveg generálás

### Tisztítás

In [ ]:
# Szükséges könyvtárak importálása
import json
import os
from google.colab import drive, files

# 1. Google Drive csatolása
drive.mount('/content/drive')

# 2. Projekt könyvtár és fájlok beállítása
project_dir = '/content/drive/MyDrive/Image'
input_filename = os.path.join(project_dir, 'desc.json')
output_filename = os.path.join(project_dir, 'cleaned_output.json')

# 3. Ellenőrzés, hogy a desc.json létezik-e
if not os.path.exists(input_filename):
    print("Hiba: A 'desc.json' nem található a projektkönyvtárban!")
    print("Kérlek, töltsd fel a fájlt, ha szükséges!")
    uploaded = files.upload()
    for fname, content in uploaded.items():
        with open(input_filename, 'wb') as f:
            f.write(content)
    print("Fájl feltöltve a projektkönyvtárba!")
else:
    print("A 'desc.json' megtalálva a projektkönyvtárban, folytatom a feldolgozást...")

# 4. JSON fájl betöltése
with open(input_filename, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 5. Promptok tisztítása függvény
def clean_prompt(prompt):
    # Perjelek cseréje szóközre
    cleaned = prompt.replace('/', ' ')
    # Zárójelek eltávolítása
    cleaned = cleaned.replace('(', '').replace(')', '')
    # Többszörös szóközök eltávolítása
    cleaned = ' '.join(cleaned.split())
    return cleaned.strip()

# 6. Minden prompt megtisztítása
cleaned_data = {}
for filename, prompt in data.items():
    cleaned_prompt = clean_prompt(prompt)
    cleaned_data[filename] = cleaned_prompt

# 7. Megtisztított JSON mentése a projektkönyvtárba
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(cleaned_data, f, ensure_ascii=False, indent=2)

# 8. Az új fájl letöltése
print("A megtisztított JSON fájl elmentve a projektkönyvtárba ('cleaned_output.json'), és letöltheted!")
files.download(output_filename)

### **SEO generation**

In [ ]:
# Szükséges könyvtárak importálása
from google.colab import drive, files
import json
import os
from transformers import pipeline

class FileManager:
    """Osztály a Google Drive fájlkezeléséhez és JSON műveletekhez."""

    def __init__(self, project_dir='/content/drive/MyDrive/Image', filename='cleaned_output.json'):
        """Inicializálja a fájlkezelőt a megadott könyvtárral és fájlnévvel."""
        self.project_dir = project_dir
        self.filename = os.path.join(project_dir, filename)
        self.data = None

    def mount_drive(self):
        """Csatolja a Google Drive-ot a Colab környezethez."""
        print("Csatolás a Google Drive-hoz...")
        drive.mount('/content/drive', force_remount=True)

    def load_json(self):
        """Betölti a JSON fájlt, ha létezik, különben hibát jelez."""
        if not os.path.exists(self.filename):
            print(f"Hiba: A '{self.filename}' nem található a projektkönyvtárban!")
            raise FileNotFoundError("Kérlek, futtasd az előző szkriptet először!")
        print(f"A '{self.filename}' megtalálva, betöltés...")
        with open(self.filename, 'r', encoding='utf-8') as f:
            self.data = json.load(f)
        return self.data

    def save_json(self):
        """Visszaírja a frissített adatokat a JSON fájlba."""
        print("Frissített fájl mentése...")
        with open(self.filename, 'w', encoding='utf-8') as f:
            json.dump(self.data, f, ensure_ascii=False, indent=2)

    def download_file(self):
        """Letölti a frissített JSON fájlt a böngészőbe."""
        print(f"A '{self.filename}' frissítve, letöltheted!")
        files.download(self.filename)

class SEOGenerator:
    """Osztály SEO leírások generálására, stílus- és technikai részletek nélkül."""

    def __init__(self, model_name='facebook/bart-large-cnn'):
        """Inicializálja a generátort a megadott modellel."""
        print(f"Betöltés a '{model_name}' modellhez a leírásokhoz...")
        self.summarizer = pipeline("summarization", model=model_name)
        # Stílus- és technikai kulcsszavak kiszűrése
        self.technical_style_terms = {
            "8k", "resolution", "cinematic", "photorealistic", "hyperrealism", "dynamic", "motion", "capture",
            "crisp", "vibrant", "colors", "high", "detail", "textures", "sharp", "focus", "depth", "field",
            "effect", "emphasize", "subject", "warm", "golden", "hour", "tones", "clean", "polished", "finish",
            "inspired", "high-fashion", "photography", "film", "stills", "ultra-detailed", "texture", "lifelike",
            "eyes", "soft", "bokeh", "effects", "lights", "background", "perfection", "style", "DSLR", "photo",
            "Fujifilm", "X-T4", "f", "2.8", "natural", "lighting", "composition", "variety", "conditions",
            "Adobe", "After", "Effects", "software", "realistic", "striking", "visible", "enhance"
        }

    def filter_technical_style_terms(self, prompt):
        """Eltávolítja a stílus- és technikai kulcsszavakat a promptból."""
        words = prompt.lower().split()
        filtered_words = [word for word in words if word not in self.technical_style_terms]
        return " ".join(filtered_words)

    def generate_seo_description(self, prompt):
        """Generál egy maximum 500 karakteres SEO leírást a promptból, stílus- és technikai részletek nélkül."""
        print(f"SEO leírás generálása a promptból (hossz: {len(prompt)} karakter)...")
        # Technikai és stílus részletek kiszűrése
        filtered_prompt = self.filter_technical_style_terms(prompt)
        print(f"Szűrt prompt: '{filtered_prompt}' ({len(filtered_prompt)} karakter)")
        # Rövid összefoglaló generálása, a fő témára fókuszálva
        summary = self.summarizer(filtered_prompt, max_length=80, min_length=20, do_sample=False)[0]['summary_text']
        seo_desc = summary.strip()[:500]
        # Redundancia eltávolítása
        words = seo_desc.split()
        seo_desc = " ".join(sorted(set(words), key=words.index))[:500]
        print(f"Generált leírás: '{seo_desc}' ({len(seo_desc)} karakter)")
        return seo_desc

class SEOTitleGenerator:
    """Osztály SEO címek generálására a promptból, a fő témára fókuszálva."""

    def __init__(self, model_name='facebook/bart-large-cnn'):
        """Inicializálja a címgenerátort a megadott modellel."""
        print(f"Betöltés a '{model_name}' modellhez a címekhez...")
        self.summarizer = pipeline("summarization", model=model_name)

    def generate_seo_title(self, prompt):
        """Generál egy maximum 100 karakteres SEO címet a promptból, a fő témára koncentrálva."""
        print(f"SEO cím generálása a promptból (hossz: {len(prompt)} karakter)...")
        short_prompt = " ".join(prompt.split()[:50])
        input_text = f"Summarize the main subject of: {short_prompt}. Focus on the key subject, keep it short, max 100 characters."
        summary = self.summarizer(input_text, max_length=15, min_length=5, do_sample=False)[0]['summary_text']
        seo_title = summary.strip()[:100]
        if len(seo_title) == 100 and not seo_title.endswith('.'):
            seo_title = seo_title[:-1] + '.'
        print(f"Generált cím: '{seo_title}' ({len(seo_title)} karakter)")
        return seo_title

def main():
    """Fő függvény az osztályok használatához és a folyamat vezérléséhez."""
    file_manager = FileManager()
    file_manager.mount_drive()
    data = file_manager.load_json()

    seo_generator = SEOGenerator()
    print("SEO leírások ellenőrzése és generálása...")
    for filename, entry in data.items():
        if isinstance(entry, dict):
            if "seo_description" not in entry or len(entry["seo_description"]) > 500:
                print(f"Frissítés szükséges: {filename}...")
                entry["seo_description"] = seo_generator.generate_seo_description(entry["prompt"])
        else:
            print(f"Átalakítás szótárrá és leírás generálása: {filename}...")
            prompt = entry
            seo_desc = seo_generator.generate_seo_description(prompt)
            data[filename] = {"prompt": prompt, "seo_description": seo_desc}

    seo_title_generator = SEOTitleGenerator()
    print("SEO címek ellenőrzése és generálása...")
    for filename, entry in data.items():
        print(f"Feldolgozás: {filename}...")
        seo_title = seo_title_generator.generate_seo_title(entry["prompt"])
        entry["seo_title"] = seo_title

    file_manager.data = data
    file_manager.save_json()
    file_manager.download_file()

if __name__ == "__main__":
    main()

### Képfelismerés

In [ ]:
from PIL import Image
import json
import os
from google.colab import drive
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
from datetime import datetime

class FileManager:
    def __init__(self, project_dir='/content/drive/MyDrive/Image', filename='cleaned_output.json'):
        self.project_dir = project_dir
        self.filename = os.path.join(project_dir, filename)
        self.data = {}

    def mount_drive(self):
        print("Mounting Google Drive...")
        drive.mount('/content/drive', force_remount=True)

    def load_json(self):
        if os.path.exists(self.filename):
            try:
                with open(self.filename, 'r', encoding='utf-8') as f:
                    self.data = json.load(f)
            except json.JSONDecodeError:
                print(f"Figyelmeztetés: '{self.filename}' érvénytelen JSON fájl.")
                self.data = {}
        else:
            print(f"'{self.filename}' nem található. Új fájl lesz létrehozva.")
        return self.data

    def save_json(self):
        try:
            with open(self.filename, 'w', encoding='utf-8') as f:
                json.dump(self.data, f, ensure_ascii=False, indent=2)
            print(f"'{self.filename}' mentve!")
        except Exception as e:
            print(f"Hiba a fájl mentése közben: {e}")

    def list_images(self):
        if not os.path.exists(self.project_dir):
            print(f"Hiba: '{self.project_dir}' könyvtár nem található.")
            return []
        return [f for f in os.listdir(self.project_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

class ImageDescriber:
    def __init__(self, processor, model):
        self.processor = processor
        self.model = model

    def load_image(self, image_path):
        try:
            return Image.open(image_path).convert("RGB")
        except Exception as e:
            print(f"Hiba a(z) '{image_path}' betöltése közben: {e}")
            return None

    def generate_description(self, image_path):
        image = self.load_image(image_path)
        if image is None:
            return "Nem sikerült betölteni a képet."
        inputs = self.processor(image, return_tensors="pt").to('cuda')
        out = self.model.generate(
            **inputs,
            max_new_tokens=500,  # Célzottan 450-500 karakter körüli leírás
            num_beams=3,         # Több variáció a részletesebb leírásért
            length_penalty=2.8,  # Erősebb ösztönzés hosszabb szövegre
            no_repeat_ngram_size=2,
            do_sample=False,     # Következetes generálás
            early_stopping=False # Ne álljon meg korán
        )
        desc = self.processor.decode(out[0], skip_special_tokens=True)
        return desc

def main():
    if not torch.cuda.is_available():
        print("Figyelmeztetés: GPU nem elérhető. A feldolgozás lassabb lehet.")
    else:
        print("GPU észlelve. GPU-t használjuk a feldolgozáshoz.")

    file_manager = FileManager()
    file_manager.mount_drive()
    data = file_manager.load_json()
    image_files = file_manager.list_images()

    if not image_files:
        print("Nincsenek képek a megadott könyvtárban.")
        return

    print(f"Talált képek: {image_files}")

    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to('cuda')
    describer = ImageDescriber(processor, model)

    for filename in image_files:
        image_path = os.path.join(file_manager.project_dir, filename)
        print(f"Feldolgozás: {filename}...")

        try:
            prompt = data.get(filename, {}).get("prompt", None)  # Csak tárolásra, nem használatra
            image_desc = describer.generate_description(image_path)
            print(f"{filename} leírása: {image_desc} (Hossz: {len(image_desc)})")

            processed_at = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            if filename not in data:
                data[filename] = {}
            data[filename].update({
                "prompt": prompt or "Nincs megadva prompt",
                "image_description": image_desc,
                "link": "",
                "processed_at": processed_at,
                "uploaded_at": ""
            })
        except Exception as e:
            print(f"Hiba a(z) '{filename}' feldolgozása közben: {e}")

    file_manager.data = data
    file_manager.save_json()

if __name__ == "__main__":
    main()

### SEO cím és leírás kulcsszavakkal

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
Képfeldolgozó és SEO-elemző szkript
Ez a szkript a Google Colab környezetben fut, és a következőket végzi:
- Képek leírásának generálása BLIP modellel (~500 karakter).
- SEO-barát cím, leírás és kulcsszavak generálása a leírás alapján.
- Releváns CTA generálása és külön mezőben tárolása.
- Az eredmények mentése JSON fájlba.
"""

from PIL import Image
import json
import os
from google.colab import drive
from transformers import BlipProcessor, BlipForConditionalGeneration, pipeline
from keybert import KeyBERT
import torch
from datetime import datetime

# Fájlkezelő osztály a Google Drive és JSON műveletekhez
class FileManager:
    def __init__(self, project_dir='/content/drive/MyDrive/Image', filename='cleaned_output.json'):
        """Inicializálja a fájlkezelőt a megadott könyvtár- és fájlnévvel."""
        self.project_dir = project_dir  # Projekt könyvtár elérési útja
        self.filename = os.path.join(project_dir, filename)  # JSON fájl teljes elérési útja
        self.data = {}  # Adatok tárolása

    def mount_drive(self):
        """Csatlakoztatja a Google Drive-ot a Colab környezetben."""
        print("Mounting Google Drive...")
        drive.mount('/content/drive', force_remount=True)

    def load_json(self):
        """Betölti a meglévő JSON fájlt, vagy létrehoz egy újat, ha nem létezik."""
        if os.path.exists(self.filename):
            try:
                with open(self.filename, 'r', encoding='utf-8') as f:
                    self.data = json.load(f)
            except json.JSONDecodeError:
                print(f"Figyelmeztetés: '{self.filename}' érvénytelen JSON fájl.")
                self.data = {}
        else:
            print(f"'{self.filename}' nem található. Új fájl lesz létrehozva.")
        return self.data

    def save_json(self):
        """Elmenti az adatokat a JSON fájlba."""
        try:
            with open(self.filename, 'w', encoding='utf-8') as f:
                json.dump(self.data, f, ensure_ascii=False, indent=2)
            print(f"'{self.filename}' mentve!")
        except Exception as e:
            print(f"Hiba a fájl mentése közben: {e}")

    def list_images(self):
        """Kilistázza a megadott könyvtárban lévő képeket (.png, .jpg, .jpeg kiterjesztéssel)."""
        if not os.path.exists(self.project_dir):
            print(f"Hiba: '{self.project_dir}' könyvtár nem található.")
            return []
        return [f for f in os.listdir(self.project_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Kép- és SEO-elemző osztály
class ImageDescriber:
    def __init__(self, processor, model):
        """Inicializálja a képelemzőt a BLIP modellel és SEO eszközökkel."""
        self.processor = processor  # BLIP processzor
        self.model = model  # BLIP modell
        # SEO és kulcsszó generáláshoz
        self.summary_generator = pipeline("summarization", model="facebook/bart-large-cnn", framework="pt")
        self.kw_model = KeyBERT()  # Kulcsszókinyerő modell

    def load_image(self, image_path):
        """Betölti a képet RGB formátumba."""
        try:
            return Image.open(image_path).convert("RGB")
        except Exception as e:
            print(f"Hiba a(z) '{image_path}' betöltése közben: {e}")
            return None

    def generate_description(self, image_path):
        """Generál egy ~500 karakteres leírást a képről a BLIP modellel."""
        image = self.load_image(image_path)
        if image is None:
            return "Nem sikerült betölteni a képet."
        inputs = self.processor(image, return_tensors="pt").to('cuda')
        out = self.model.generate(
            **inputs,
            max_new_tokens=300,  # Célzottan ~500 karakter
            num_beams=4,
            length_penalty=2.7,  # Erősebb ösztönzés hosszabb szövegre
            no_repeat_ngram_size=3,  # Zagyvaság csökkentése
            do_sample=False,
            early_stopping=False
        )
        desc = self.processor.decode(out[0], skip_special_tokens=True)
        if len(desc) > 500:
            desc = desc[:500]  # Maximum 500 karakter
        return desc

    def generate_relevant_cta(self, image_description):
        """Generál egy releváns CTA-t a leírás kulcsszavai alapján, félkövérrel és emojikkal."""
        desc_lower = image_description.lower()
        cta_options = {
            "woman": "Get inspired by this style!",
            "man": "Check out this look!",
            "dog": "Meet this adorable pet!",
            "cat": "Meet this cute cat!",
            "street": "Explore this urban vibe!",
            "sun": "Enjoy this sunny moment!",
            "leaves": "Admire this autumn scene!",
            "salon": "Visit this pet salon!"
        }
        for keyword, cta_text in cta_options.items():
            if keyword in desc_lower:
                return f"📌 **{cta_text}** ✨"
        return f"📌 **Discover this scene!** ✨"  # Alapértelmezett

    def generate_seo_elements(self, image_description):
        """Generál SEO címet, leírást és kulcsszavakat az image_description alapján."""
        # SEO cím (max 100 karakter)
        summary_input = f"Summarize for SEO title: {image_description}"
        summary = self.summary_generator(summary_input, max_length=25, min_length=5, do_sample=False)[0]['summary_text']
        seo_title = f"{summary[:80].strip()} - Pin It!"
        if len(seo_title) > 100:
            seo_title = seo_title[:97] + "..."

        # SEO leírás (max 160 karakter)
        desc_input = f"Summarize for SEO description: {image_description}"
        seo_desc = self.summary_generator(desc_input, max_length=90, min_length=20, do_sample=False)[0]['summary_text']
        if len(seo_desc) > 160:
            seo_desc = seo_desc[:157] + "..."
        if "kitchen" in seo_desc.lower() and "salon" in image_description.lower():
            seo_desc = seo_desc.replace("kitchen", "salon")

        # 5 releváns kulcsszó a seo_description-ból
        keywords = [kw[0] for kw in self.kw_model.extract_keywords(seo_desc, keyphrase_ngram_range=(1, 2), stop_words='english', top_n=12)[:5]]

        return {
            "seo_title": seo_title,
            "seo_description": seo_desc,
            "keywords": keywords
        }

def main():
    """Fő függvény, amely a teljes feldolgozási folyamatot vezérli."""
    # GPU elérhetőség ellenőrzése
    if not torch.cuda.is_available():
        print("Figyelmeztetés: GPU nem elérhető. A feldolgozás lassabb lehet.")
    else:
        print("GPU észlelve. GPU-t használjuk a feldolgozáshoz.")

    # Fájlkezelő inicializálása és Drive csatlakoztatása
    file_manager = FileManager()
    file_manager.mount_drive()
    data = file_manager.load_json()
    image_files = file_manager.list_images()

    # Képfeldolgozás ellenőrzése
    if not image_files:
        print("Nincsenek képek a megadott könyvtárban.")
        return

    print(f"Talált képek: {image_files}")

    # Modell inicializálása
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to('cuda')
    describer = ImageDescriber(processor, model)

    # Képfeldolgozás és SEO elemzés
    for filename in image_files:
        image_path = os.path.join(file_manager.project_dir, filename)
        print(f"Feldolgozás: {filename}...")

        try:
            prompt = data.get(filename, {}).get("prompt", None)  # Csak tárolásra
            image_desc = describer.generate_description(image_path)
            cta = describer.generate_relevant_cta(image_desc)
            total_length = len(image_desc) + len(cta)
            print(f"{filename} leírása: {image_desc} (Hossz: {len(image_desc)})")
            print(f"{filename} CTA: {cta} (Hossz: {len(cta)}, Összesen: {total_length})")

            # SEO elemek generálása
            seo_elements = describer.generate_seo_elements(image_desc)

            processed_at = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            if filename not in data:
                data[filename] = {}
            data[filename].update({
                "prompt": prompt or "Nincs megadva prompt",
                "image_description": image_desc,
                "seo_title": seo_elements["seo_title"],
                "seo_description": seo_elements["seo_description"],
                "keywords": seo_elements["keywords"],
                "cta": cta,
                "link": "",
                "processed_at": processed_at,
                "uploaded_at": ""
            })
        except Exception as e:
            print(f"Hiba a(z) '{filename}' feldolgozása közben: {e}")

    # Eredmények mentése
    file_manager.data = data
    file_manager.save_json()

if __name__ == "__main__":
    main()

### Paraméterezhető SEO cím és leírás kulcsszavakkal

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
Képfeldolgozó és SEO-elemző szkript
Ez a szkript a Google Colab környezetben fut, és a következőket végzi:
- Képek leírásának generálása BLIP modellel (~500 karakter).
- SEO-barát cím, leírás és kulcsszavak generálása a leírás alapján.
- Releváns CTA generálása és külön mezőben tárolása.
- Az eredmények mentése JSON fájlba.

A finomhangolható paraméterek modellenként külön szótárakban vannak definiálva a script elején,
hogy könnyen módosíthatóak legyenek a kimenet testreszabásához.
"""

from PIL import Image
import json
import os
from google.colab import drive
from transformers import BlipProcessor, BlipForConditionalGeneration, pipeline
from keybert import KeyBERT
import torch
from datetime import datetime

# Paraméterek konfigurációja
# BLIP modell paraméterei (képleírás generáláshoz)
BLIP_CONFIG = {
    "max_new_tokens": 300,  # Maximális új tokenek száma a generáláshoz (~500 karakter)
    # Magyarázat: Meghatározza a generálandó új tokenek maximális számát. Egy token ~1.5-2 karakter,
    # így ez a leírás hosszát szabályozza. Növelés (pl. 320) hosszabb, csökkentés (pl. 280) rövidebb leírást eredményez.
    "num_beams": 4,  # Beam search száma a legjobb szekvenciák kiválasztásához
    # Magyarázat: Minél nagyobb, annál jobb koherencia, de lassabb. 3 stabil, 5 részletgazdag, de terhelő lehet.
    "length_penalty": 2.7,  # Hossz büntetési tényező
    # Magyarázat: >1 hosszabb, <1 rövidebb szöveget ösztönöz. 2.5-3.0 hosszabb, 2.0 alatti rövidebb leírást ad.
    "no_repeat_ngram_size": 4,  # Ismétlődő n-gramok száma, amit el kell kerülni
    # Magyarázat: Csökkenti a redundanciát. 2 rugalmas, 4 szigorúbb, 3 kompromisszum.
    "do_sample": False,  # Determinisztikus vs. sztochasztikus generálás
    # Magyarázat: False konzisztens, True kreatívabb, de instabil. True esetén temperature finomhangolható.
    "early_stopping": False  # Korai leállítás a generálásnál
    # Magyarázat: False a max_new_tokens-ig megy, True korán leáll, ha befejezettnek ítéli.
}

# BART modell paraméterei (SEO elemek generálásához)
BART_CONFIG = {
    "title_max_length": 25,  # SEO cím maximális tokenhossza
    # Magyarázat: A generált cím token maximális száma (~100 karakter). 20-30 között finomítható.
    "title_min_length": 5,  # SEO cím minimális tokenhossza
    # Magyarázat: Biztosítja a minimális tartalmat. 5-10 között állítható.
    "desc_max_length": 100,  # SEO leírás maximális tokenhossza
    # Magyarázat: A generált leírás token maximális száma (~160 karakter). 80-120 között finomítható.
    "desc_min_length": 20,  # SEO leírás minimális tokenhossza
    # Magyarázat: Minimális tartalom biztosítása. 15-30 között állítható.
    "do_sample": False  # Determinisztikus vs. sztochasztikus generálás
    # Magyarázat: False konzisztens, True kreatívabb. True esetén temperature (pl. 0.7) finomhangolható.
    # temperature alapértelmezett 1.0, alacsonyabb (0.8) konzervatívabb, magasabb (1.2) kreatívabb.
}

class FileManager:
    def __init__(self, project_dir='/content/drive/MyDrive/Image', filename='cleaned_output.json'):
        """Inicializálja a fájlkezelőt a megadott könyvtár- és fájlnévvel."""
        self.project_dir = project_dir
        self.filename = os.path.join(project_dir, filename)
        self.data = {}

    def mount_drive(self):
        """Csatlakoztatja a Google Drive-ot a Colab környezetben."""
        print("Mounting Google Drive...")
        drive.mount('/content/drive', force_remount=True)

    def load_json(self):
        """Betölti a meglévő JSON fájlt, vagy létrehoz egy újat, ha nem létezik."""
        if os.path.exists(self.filename):
            try:
                with open(self.filename, 'r', encoding='utf-8') as f:
                    self.data = json.load(f)
            except json.JSONDecodeError:
                print(f"Figyelmeztetés: '{self.filename}' érvénytelen JSON fájl.")
                self.data = {}
        else:
            print(f"'{self.filename}' nem található. Új fájl lesz létrehozva.")
        return self.data

    def save_json(self):
        """Elmenti az adatokat a JSON fájlba."""
        try:
            with open(self.filename, 'w', encoding='utf-8') as f:
                json.dump(self.data, f, ensure_ascii=False, indent=2)
            print(f"'{self.filename}' mentve!")
        except Exception as e:
            print(f"Hiba a fájl mentése közben: {e}")

    def list_images(self):
        """Kilistázza a megadott könyvtárban lévő képeket (.png, .jpg, .jpeg kiterjesztéssel)."""
        if not os.path.exists(self.project_dir):
            print(f"Hiba: '{self.project_dir}' könyvtár nem található.")
            return []
        return [f for f in os.listdir(self.project_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

class ImageDescriber:
    def __init__(self, processor, model):
        """Inicializálja a képelemzőt a BLIP modellel és SEO eszközökkel."""
        self.processor = processor
        self.model = model
        self.summary_generator = pipeline("summarization", model="facebook/bart-large-cnn", framework="pt")
        self.kw_model = KeyBERT()

    def load_image(self, image_path):
        """Betölti a képet RGB formátumba."""
        try:
            return Image.open(image_path).convert("RGB")
        except Exception as e:
            print(f"Hiba a(z) '{image_path}' betöltése közben: {e}")
            return None

    def generate_description(self, image_path):
        """Generál egy ~500 karakteres leírást a képről a BLIP modellel."""
        image = self.load_image(image_path)
        if image is None:
            return "Nem sikerült betölteni a képet."
        inputs = self.processor(image, return_tensors="pt").to('cuda')
        out = self.model.generate(
            **inputs,
            max_new_tokens=BLIP_CONFIG["max_new_tokens"],
            num_beams=BLIP_CONFIG["num_beams"],
            length_penalty=BLIP_CONFIG["length_penalty"],
            no_repeat_ngram_size=BLIP_CONFIG["no_repeat_ngram_size"],
            do_sample=BLIP_CONFIG["do_sample"],
            early_stopping=BLIP_CONFIG["early_stopping"]
        )
        desc = self.processor.decode(out[0], skip_special_tokens=True)
        if len(desc) > 500:
            desc = desc[:500]
        return desc

    def generate_relevant_cta(self, image_description):
        """Generál egy releváns CTA-t a leírás kulcsszavai alapján, félkövérrel és emojikkal."""
        desc_lower = image_description.lower()
        cta_options = {
            "woman": "Get inspired by this style!",
            "man": "Check out this look!",
            "dog": "Meet this adorable pet!",
            "cat": "Meet this cute cat!",
            "street": "Explore this urban vibe!",
            "sun": "Enjoy this sunny moment!",
            "leaves": "Admire this autumn scene!",
            "salon": "Visit this pet salon!"
        }
        for keyword, cta_text in cta_options.items():
            if keyword in desc_lower:
                return f"📌 **{cta_text}** ✨"
        return f"📌 **Discover this scene!** ✨"

    def generate_seo_elements(self, image_description):
        """Generál SEO címet, leírást és kulcsszavakat az image_description alapján."""
        summary_input = f"Summarize for SEO title: {image_description}"
        summary = self.summary_generator(summary_input,
                                        max_length=BART_CONFIG["title_max_length"],
                                        min_length=BART_CONFIG["title_min_length"],
                                        do_sample=BART_CONFIG["do_sample"])[0]['summary_text']
        seo_title = f"{summary[:80].strip()} - Pin It!"
        if len(seo_title) > 100:
            seo_title = seo_title[:97] + "..."

        desc_input = f"Summarize for SEO description: {image_description}"
        seo_desc = self.summary_generator(desc_input,
                                         max_length=BART_CONFIG["desc_max_length"],
                                         min_length=BART_CONFIG["desc_min_length"],
                                         do_sample=BART_CONFIG["do_sample"])[0]['summary_text']
        if len(seo_desc) > 160:
            seo_desc = seo_desc[:157] + "..."
        if "kitchen" in seo_desc.lower() and "salon" in image_description.lower():
            seo_desc = seo_desc.replace("kitchen", "salon")

        keywords = [kw[0] for kw in self.kw_model.extract_keywords(seo_desc,
                                                                  keyphrase_ngram_range=(1, 2),
                                                                  stop_words='english',
                                                                  top_n=15)[:5]]

        return {
            "seo_title": seo_title,
            "seo_description": seo_desc,
            "keywords": keywords
        }

def main():
    """Fő függvény, amely a teljes feldolgozási folyamatot vezérli."""
    if not torch.cuda.is_available():
        print("Figyelmeztetés: GPU nem elérhető. A feldolgozás lassabb lehet.")
    else:
        print("GPU észlelve. GPU-t használjuk a feldolgozáshoz.")

    file_manager = FileManager()
    file_manager.mount_drive()
    data = file_manager.load_json()
    image_files = file_manager.list_images()

    if not image_files:
        print("Nincsenek képek a megadott könyvtárban.")
        return

    print(f"Talált képek: {image_files}")

    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to('cuda')
    describer = ImageDescriber(processor, model)

    for filename in image_files:
        image_path = os.path.join(file_manager.project_dir, filename)
        print(f"Feldolgozás: {filename}...")

        try:
            prompt = data.get(filename, {}).get("prompt", None)
            image_desc = describer.generate_description(image_path)
            cta = describer.generate_relevant_cta(image_desc)
            total_length = len(image_desc) + len(cta)
            print(f"{filename} leírása: {image_desc} (Hossz: {len(image_desc)})")
            print(f"{filename} CTA: {cta} (Hossz: {len(cta)}, Összesen: {total_length})")

            seo_elements = describer.generate_seo_elements(image_desc)

            processed_at = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            if filename not in data:
                data[filename] = {}
            data[filename].update({
                "prompt": prompt or "Nincs megadva prompt",
                "image_description": image_desc,
                "seo_title": seo_elements["seo_title"],
                "seo_description": seo_elements["seo_description"],
                "keywords": seo_elements["keywords"],
                "cta": cta,
                "link": "",
                "processed_at": processed_at,
                "uploaded_at": ""
            })
        except Exception as e:
            print(f"Hiba a(z) '{filename}' feldolgozása közben: {e}")

    file_manager.data = data
    file_manager.save_json()

if __name__ == "__main__":
    main()